In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [38]:
import numpy as np
import argparse
import ast
import time
import utils
import matplotlib.pyplot as plt
import logging

In [41]:
logging.basicConfig(level=logging.INFO)

In [49]:
# import sys
# sys.path.append("/Users/hangjianli/Documents/research/change_point/gflsegpy/")

In [50]:
# import gflsegpy as seg  

In [9]:
def plot_theta(theta, filename, thr=3, change_pts=None):
    """Plot """
    t, p = theta.shape
    fig, axx = plt.subplots(1, 1, figsize=(21, 5))
    norm_diff = np.linalg.norm(np.diff(theta, axis=0), ord=2, axis=1)
    estimated_change_pts = np.arange(t-1)[norm_diff > thr] + 1
    print(f"The estimated change points are {estimated_change_pts}")
    axx.plot(np.arange(1, t), norm_diff)
    if change_pts is not None:
        for i, cp in enumerate(change_pts):
            axx.vlines(x = cp, ymin=0, ymax=max(norm_diff), ls = '--', color = 'r',
                       label='True change points' if i == 0 else "")
    axx.set_title("l2-norm difference in theta between t and t+1", fontsize=30)
    fig.tight_layout()
    plt.legend(fontsize=20)
    plt.savefig(filename, bbox_inches='tight')
    plt.close(fig)


In [34]:
# current directory should be '/Users/hangjianli/Documents/research/change_point/python'
outdir = "../output/scenario1" 

### Load setup info and data

In [43]:
logging.info("Loading network statistics and edge data...")
experiment_dir = outdir + ('' if outdir.endswith('/') else '/')

with open(experiment_dir + 'args/args.txt', 'r') as f: # comes from make_data.py
    sim_args_str = f.read()

sim_args = ast.literal_eval(sim_args_str)
p = len(sim_args['form_terms']) + len(sim_args['diss_terms'])

H = np.loadtxt(experiment_dir + sim_args['data_name'] + '_H.txt') # suff stats
y = np.loadtxt(experiment_dir + sim_args['data_name'] + '_y.txt') # networks

t = H.shape[0]
H = H.reshape((t, -1, p)) # t x n^2(E) x p
n = np.sqrt(H.shape[1]).astype(int)
logging.info(f"Data has dimension (t, n, p): ({t}, {n}, {p})")

INFO:root:Loading network statistics and edge data...
INFO:root:Data has dimension (t, n, p): (150, 20, 4)


In [46]:
args = {
    'outdir': "../output/newsim1",
    'lam': 10,
    'admm_alpha': 1,
    'solver': 'newton',
    'rel_tol': 1e-7, #Stopping criterion for change in theta
    'max_steps': 100,
    'max_steps_newton': 100,
    'conv_tol': 1e-5,
    'gd_lr': 0.01,
    'gd_epochs': 100,
    'gfl_tol':1e-5,
    'gfl_maxit':1000,
    'true_cp': None, #True change points (for plot).
    "cp_thr": 3, #"Threshold for estimating change points
}


### Save args and define output files

In [48]:
# Get the output filenames
result_dir = utils.make_directory(experiment_dir, 'results')
args_dir = utils.make_directory(experiment_dir, 'args')
utils.save_args(args, args_dir + 'args_model.txt')

theta_outfile = result_dir + 'theta_' + sim_args['data_name'] + ".txt"
u_outfile = result_dir + 'u_' + sim_args['data_name'] + ".txt"
z_outfile = result_dir + 'z_' + sim_args['data_name'] + ".txt"
theta_plot_dir = result_dir + 'est_theta_diff.png'

### Run STERGM model

In [ ]:
print('Initialize STERGM model...')
model = mple_learn.STERGMGraph(
    lam=args.lam,
    admm_alpha=args.admm_alpha,
    rel_tol=args.rel_tol,
    max_steps=args.max_steps,
    newton_max_steps=args.max_steps_newton,
    converge_tol=args.conv_tol,
    gd_lr=args.gd_lr,
    gd_epochs=args.gd_epochs,
    gfl_tol=args.gfl_tol,
    gfl_maxit=args.gfl_maxit,
    verbose=sim_args["verbose"])

model.load_data(H, y, t, p)

print("Run mple to estimate theta...")
start = time.time()
res = model.mple(solver=args.solver)
end = time.time()
print(f"[INFO] MPLE finished in: {end - start}.")
theta = res['theta']
u = res['u']
z = res['z']

print("Saving estimated parameters...")
np.savetxt(theta_outfile, theta)
np.savetxt(u_outfile, u)
np.savetxt(z_outfile, z)

plot_theta(theta, theta_plot_dir, args.cp_thr, args.true_cp)
print("Done!")